In [ ]:
# !pip install --quiet langchain langgraph langchain_community tiktoken chroma chromadb


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# !pip install torch torchvision torchaudio

  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.2 MB/s eta 0:00:00a 0:00:01
Using cached setuptools-75.8.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import torch

In [6]:
# if torch.cuda.is_available():
#     print(f"GPU available: {torch.cuda.get_device_name(0)}")
#     DEVICE = "cuda"
# else:
#     print("No GPU available, using CPU")
#     DEVICE = "cpu"

if torch.backends.mps.is_available():
    print(f"GPU available: MPS")
    DEVICE = "mps"
else:
    print("GPU isn't available, using CPU")

GPU available: MPS


In [ ]:
# !pip install --quiet sentence-transformers


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [12]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

embeddings = HuggingFaceBgeEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={"device": DEVICE}
)

/Users/vishals/Desktop/Pharma-Knowledge-asst/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import json
import os
import chromadb

from tqdm.auto import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from chromadb.config import Settings


print("Starting to read Json Files:")
docs = list()
data_dir = "datasets/microlabs_usa"
file_count = len([f for f in os.listdir(data_dir) if f.endswith('.json')])

for filename in tqdm(os.listdir(data_dir), desc="Loading files"):
        if filename.endswith(".json"):
            with open(os.path.join(data_dir, filename), 'r', encoding='utf-8') as f:
                data = json.load(f)
                text = json.dumps(data, indent=2)
                docs.append(text)

print("Starting the text splitting....")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.create_documents(docs)
total_chunks = len(splits)
print(f"Created {total_chunks} text chunks")

print("Creating ChromaDB database...")
chroma_client = chromadb.Client(Settings(
        anonymized_telemetry=False,
        is_persistent=True,
        persist_directory="chroma_db"
    ))

batch_size = 64
total_batches = (total_chunks + batch_size - 1)
print(f"Processing embeddings in {total_batches}")
with tqdm(total=total_chunks, desc="Creating embeddings...") as progbar:
    for i in range(0, len(splits), batch_size):
        batch = splits[i:i + batch_size]

        if i == 0:
            vector_store = Chroma.from_documents(
                  documents=batch,
                  embedding=embeddings,
                  client=chroma_client,
                  collection_name="pharma-kb"
            )
        else:
          vector_store.add_documents(documents=batch)
        progbar.update(len(batch))
        if DEVICE == "cuda" and i % (batch_size * 4) == 0:
          torch.cuda.empty_cache()
print("ChromaDB database created successfully")

Starting to read Json Files:


Loading files: 100%|██████████| 45/45 [00:00<00:00, 192.46it/s]


Starting the text splitting....
Created 119664 text chunks
Creating ChromaDB database...
Processing embeddings in 119727


Creating embeddings...: 100%|██████████| 119664/119664 [13:26<00:00, 148.35it/s]

ChromaDB database created successfully


In [ ]:
# def _initialize_vectorstore():
#         """Initialize ChromaDB vectorstore"""
#         import chromadb
#         from chromadb.config import Settings

#         chroma_client = chromadb.Client(Settings(
#             anonymized_telemetry=False,
#             is_persistent=True,
#             persist_directory="chroma_db"
#         ))

#         try:
#             collection = chroma_client.get_collection(name="pharma-kb")
#             vectorstore = Chroma(
#                 client=chroma_client,
#                 collection_name="pharma-kb",
#                 embedding_function=embeddings
#             )
#             print(f"Loaded existing collection with {collection.count()} documents.")
#         except ValueError:
#             print("No existing collection found. Please create the vectorstore first.")

In [21]:
_initialize_vectorstore()

/var/folders/5j/0l92cdk14pl6k0dfrk9bbbkc0000gn/T/ipykernel_7841/2433487363.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Loaded existing collection with 119664 documents.
